In [1]:
import numpy as np
import pandas as pd
import os, sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import utils
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance

In [2]:
def train_data():
    X_train = pd.read_pickle('../1.Feature generation/data-X_train.pkl.bz2', compression='bz2')
    y_train = pd.read_pickle('../1.Feature generation/data-y_train.pkl.bz2', compression='bz2')
    X_test  = pd.read_pickle('../1.Feature generation/data-X_test.pkl.bz2', compression='bz2')
    y_test  = pd.read_pickle('../1.Feature generation/data-y_test.pkl.bz2', compression='bz2')

    return X_train, X_test, y_train, y_test

gridSearch = False

# Building Random Forest

In [3]:
X_train, X_test, y_train, y_test = train_data()

if gridSearch:
    n_est = [int(x) for x in np.linspace(start = 1, stop = 1000, num = 10)]
    max_d = [int(x) for x in np.linspace(10, 1000, num = 10)]
    max_d.append(None)
    parameters = {'n_estimators': n_est, 'max_features': [None,'sqrt'], 'bootstrap': [True], 'max_depth': max_d, 'min_samples_split': [2,5], 'min_samples_leaf': [1, 2, 4]}
    
else :
    parameters = {'n_estimators': [445], 'min_samples_split': [2], 'min_samples_leaf': [1], 'max_features': [None], 'max_depth': [450], 'bootstrap': [True]}

model = RandomForestRegressor()
grid = RandomizedSearchCV(model, parameters, n_iter = 100, cv = 5, verbose=1, random_state=1, n_jobs = -1)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/home/miriam/.local/lib/python3.8/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 1 is smaller than n_iter=100. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [4]:
print('- Best Params:', grid.best_params_)
pd.DataFrame(grid.cv_results_).loc[:,['params', 'mean_test_score','rank_test_score']].sort_values(by='rank_test_score')

- Best Params: {'n_estimators': 445, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 450, 'bootstrap': True}


,params,mean_test_score,rank_test_score
0,"{'n_estimators': 445, 'min_samples_split': 2, ...",0.839107,1


In [5]:
%store grid

Stored 'grid' (RandomizedSearchCV)
